In [1]:
import sys
sys.path.append('/Users/bradh/hed-dlg/')

import numpy as np

import blocks
from blocks.bricks import Linear
from blocks.bricks.recurrent import GatedRecurrent, Fork, LSTM
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform

import theano
from theano import tensor as T

ImportError: No module named numpy

# Encoder RNN

In [33]:
#dim = original data dimension (columns)
dim = 5
rnnType = 'gru'
X = T.tensor3('inputs') #X has dimensions of input data (10,15,5)
linewt_init = Uniform(width=0.08)
rnnwt_init = IsotropicGaussian(0.05)
rnnbias_init = Constant(0.0)

if rnnType == 'gru':
    rnn = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 4

###ICLR suggestion -> don't use bias in RNNs
###RECURRENT LAYER
fork = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = rnnbias_init)
data1, data2 = fork.apply(X)

if rnnType == 'gru':
    hEnc = rnn.apply(data1, data2)[:,-1] #the [:,-1] gets the last hidden state for each obs in minibatch
else:
    hinit, _ = rnn.apply(data2)
    hEnc = hinit[:,-1]

#get weights initialized. without weights are nans.
fork.initialize()
rnn.initialize()

###ALT RNN LAYER
def initialize(to_init):
    for bricks in to_init:
        bricks.weights_init = initialization.Uniform(width=0.08)
        bricks.biases_init = initialization.Constant(0)
        bricks.initialize()

def gru_layer(dim, h, n):
    fork = Fork(output_names=['linear' + str(n), 'gates' + str(n)],
                name='fork' + str(n), input_dim=dim, output_dims=[dim, dim * 2])
    gru = GatedRecurrent(dim=dim, name='gru' + str(n))
    initialize([fork, gru])
    linear, gates = fork.apply(h)
    return gru.apply(linear, gates)


def lstm_layer(dim, h, n):
    linear = Linear(input_dim=dim, output_dim=dim * 4, name='linear' + str(n))
    lstm = LSTM(dim=dim, name='lstm' + str(n))
    initialize([linear, lstm])
    return lstm.apply(linear.apply(h))


In [36]:
#test
dataTest = theano.function([X], hEnc, allow_input_downcast=True)

In [42]:
#test
fakeData = np.random.rand(10,15,5)
dataTest(fakeData)

(10, 5)

# Context RNN

In [ ]:

if rnnType == 'gru':
    rnnContext = GatedRecurrent(dim=numMiniSessions, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnnContext = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 4

###ICLR suggestion -> don't use bias in RNNs
###RECURRENT LAYER
forkContext = Fork(output_names=['linearContext', 'gatesContext'],
            name='forkContext', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = rnnbias_init)
data3, data4 = forkContext.apply(hEnc)

if rnnType == 'gru':
    hContext = rnnContext.apply(data3, data4)[:,-1] #the [:,-1] gets the last hidden state for each obs in minibatch
else:
    hinitContext, _ = rnnContext.apply(data4)
    hContext = hinit[:,-1]

#get weights initialized. without weights are nans.
forkContext.initialize()
rnnContext.initialize()

# Decoder RNN (also pretraining)

In [ ]:
#input is hContext
#we need original input for softmax
#figure out how cost is calculated
#pray

In [ ]:

mu = theano.shared
sigma = theano.shared

# Putting it all together

In [ ]:
#output will be same dimension as input
test = theano.function([X], h, allow_input_downcast=True)

In [ ]:
##DATA for GRU
#data.shape = (a,b,c)
#a = number in minibatch
#b = timesteps
#c = features in one timestep (eg. size of dictionary, number chars, embedding space)
fakeData = np.random.rand(10, 15, 5)



In [ ]:
test(fakeData)

In [ ]:
linear = Linear(input_dim = 5, output_dim = 5, weights_init=Identity(2), biases_init=Constant(0.0))

In [ ]:
lineout = linear.apply(X)
linear.initialize()

In [ ]:
lineTest = theano.function([X], lineout, allow_input_downcast=True)

In [ ]:
lineTest(fakeData[0:1])

In [ ]:
lineTest.get_shared()[0].get_value()

In [ ]:
dim = 5
fork = Fork(output_names=['linear', 'gates'],
                name='fork', input_dim=dim, output_dims=[dim, dim * 2], 
                weights_init = Uniform(width=0.08), biases_init = Constant(0.0))

In [ ]:
fork.initialize()

In [ ]:
forkOut = fork.apply(X)

In [ ]:
forkTest = theano.function([X], forkOut, allow_input_downcast=True)

In [ ]:
forkTest(fakeData)

In [ ]:
forkTest.get_shared()[2].get_value().shape

# Converting to CPU

In [ ]:
#GPU TO CPU conversion
#Now get the weights from the test function. These weights will be numpy arrays
w1 = test.get_shared()[0].get_value()

#Here the weights are going to be set to the numpy arrays taken from the GPU predict function
input_linear.parameters[0].set_value(w1)

In [ ]:
test.get_shared()[2].get_value().shape

In [ ]:
chars = '1234567890abcdefghijklmnopqrstuvwxyz'
words = ['']

# Scratchpad